# 9. Evaluation und Interpreation
## Anforderungen an Projektumsetzung: Evaluation und Interpretation
---
**AUFGABE:**

- Setzen Sie die Aufgabenstellung für ein Prädiktionsmodell um -- wahlweise für Ihr Klassifikationsmodell oder Ihr Regressionsmodell aus den Anforderungen aus den entsprechenden Kapiteln.
- Definieren Sie für Ihr Modell die Frequenzbaseline bzw. die Mittelwertsbaseline.
- Definieren Sie für Ihr Modell eine einfache Vergleichsbaseline.
- Prüfen Sie mittels einer Lernkurve, ob Ihr Modell zu Over- oder Underfitting neigt und evaluieren Sie entsprechend des Ergebnisses ein mächtigeres oder weniger mächtiges Modell. Wenn Ihr Modell weder Over- noch Underfitting zeigt: Herzlichen Glückwunsch, es ist nichts weiter zu tun.
- Interpretieren Sie Ihr Modell: Entweder mit Hilfe von LIME oder bei transparenten Algorithmen aufgrund des gelernten Modells selber.
-----
### Mittelwertsbaseline mit prädiktiver Regression

In [18]:
# Imports
from sklearn.datasets import load_boston
import sklearn.ensemble
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import classification_report
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
from sklearn.ensemble import RandomForestRegressor

In [19]:
apps = pd.read_csv("Daten/Google-Playstore_Edit2.csv")

In [20]:
apps['Free']             = apps['Free'].astype(float)
apps['Ad Supported']     = apps['Ad Supported'].astype(float)
apps['Editors Choice']   = apps['Editors Choice'].astype(float)
apps['In App Purchases'] = apps['In App Purchases'].astype(float)
apps['Maximum Installs'] = apps['Maximum Installs'].astype(float)

In [21]:
# Um den OLS anwenden zu können, müssen die notwenidigen Features numerisch umcodiert werden
# Und alle Features löschen, die nur Unique-Werte besitzen, zu viel Rechenkapazität benötigen und für eine lineare Regression irrelvant sind

apps.drop(columns=['App Name', 'App Id', 'Installs', 'Minimum Installs', 'Developer Id', 'Developer Website', 'Minimum Android', 'Developer Email', 'Privacy Policy', 'Released', 'Scraped Time', 'Last Updated', 'Currency', 'Category', 'Editors Choice'], inplace=True)
#apps = apps.dropna()

numerical_cols = list(apps.select_dtypes(include="float").columns)
categorical_cols = list(apps.select_dtypes(include="object").columns)

object_encode = pd.get_dummies(apps[categorical_cols])

In [22]:
numeric = pd.concat([apps[numerical_cols], object_encode], axis = 1)
numeric = numeric.dropna()
numeric.head()

,Rating,Rating Count,Maximum Installs,Free,Price,Size,Ad Supported,In App Purchases,Released Year,Content Rating_Adults only 18+,Content Rating_Everyone,Content Rating_Everyone 10+,Content Rating_Mature 17+,Content Rating_Teen,Content Rating_Unrated
0,5.0,20.0,179.0,1.0,0.0,6.7,1.0,0.0,2020.0,0,1,0,0,0,0
1,5.0,8.0,152.0,1.0,0.0,71.0,1.0,0.0,2020.0,0,0,0,0,1,0
2,5.0,8.0,1338.0,1.0,0.0,2.4,0.0,0.0,2020.0,0,1,0,0,0,0
3,5.0,22.0,145.0,1.0,0.0,50.0,0.0,0.0,2020.0,0,0,0,0,1,0
4,5.0,13.0,69.0,1.0,0.0,18.0,1.0,1.0,2019.0,0,1,0,0,0,0


In [23]:
numeric.rename(columns={'Maximum Installs':'Maximum_Installs', 'Rating Count':'Rating_Count', 'Ad Supported':'Ad_Supported', 'In App Purchases':'In_App_Purchases', 'Released Year':'Released_Year', 'Content Rating_Adults only 18+':'Content_Rating_Adults_only_over18', 'Content Rating_Everyone':'Content_Rating_Everyone', 'Content Rating_Everyone 10+':'Content_Rating_Everyone_over10', 'Content Rating_Mature 17+':'Content_Rating_Mature_over17', 'Content Rating_Teen':'Content_Rating_Teen', 'Content Rating_Unrated':'Content_Rating_Unrated'}, inplace=True)

In [24]:
cols_ratio = ['Rating', 'Rating_Count','Price']
cols_target = ['Maximum_Installs']

dataSet_ratio = numeric.loc[:, cols_ratio]
dataSet_target = numeric[cols_target]

In [25]:
X = dataSet_ratio[['Price']]
Y = numeric[cols_target]

In [26]:
from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [27]:
# Modell trainieren

clf = LinearRegression()
clf.fit(X_train, y_train)

# Modell evaluieren

test_predict = clf.predict(X_test)
print("RMSE der Modellvorhersagen: ", mean_squared_error(y_test, test_predict))

# Mittelwertsvorhersage: Vorhersage ist ein Array in Länge der Testdaten, der an jeder Position den
# Mittelwert der Zieldaten aus dem Trainingsset enthält

test_predict_mean = np.full(len(y_test),y_train.mean())

print("RMSE der Mittelwertsbaseline: ", mean_squared_error(y_test, test_predict_mean))

RMSE der Modellvorhersagen:  724680702771171.6
RMSE der Mittelwertsbaseline:  724680871424443.8


### Einfache Vergleichsbaseline

In [28]:
#linearen Lerner benutzen, komplexeren regressor als LinearRegression, 
# um einen Vergleich aus 3 unterschiedlichen Regressionsergebnissen ziehen zu können
regr = RandomForestRegressor(max_depth=10, random_state=0) 
regr.fit(X_train, y_train)

test_label = regr.predict(X_test)

#regression statt klassifikation
print("RMSE der Modellvorhersagen: ", mean_squared_error(y_test, test_label))

/tmp/ipykernel_2784/2273214653.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(X_train, y_train)


RMSE der Modellvorhersagen:  724678701145346.0


-----
- das trainierte Modell ist minimal besser als das naive Baseline (Mittelwertsbaseline)
- aber die einfache Vergleichsbaseline mit einem RandomForestRegressor ist wiederum besser als der tranierte Modell
-----

### Lernkurve

In [20]:
def plot_learning_curves(model, X_train,y_train, X_test, y_test):
    train_fs, test_fs = [], []
    for m in range(100, X_train.shape[0],100):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_test_predict = model.predict(X_test)
        train_fs.append(mean_squared_error(y_train[:m], y_train_predict))
        test_fs.append(mean_squared_error(y_test, y_test_predict))
    plt.plot(train_fs, "r-+", linewidth=2, label="train")
    plt.plot(test_fs, "b-", linewidth=3, label="test")
    plt.legend()
    plt.xlabel('Index der Trainingsiteration')
    plt.ylabel('F-Score des Lerners')

In [ ]:
# Lerner vorbereiten: den besseren Regressor benutzen 
regr = RandomForestRegressor(max_depth=10, random_state=0) #durch Regressionslener ersetzen

# Lernkurve plotten - läuft eine Weile!
plot_learning_curves(regr, X_train, y_train, X_test, y_test)

/tmp/ipykernel_1044/2946831180.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train[:m], y_train[:m])
/tmp/ipykernel_1044/2946831180.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train[:m], y_train[:m])
/tmp/ipykernel_1044/2946831180.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train[:m], y_train[:m])
/tmp/ipykernel_1044/2946831180.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train[:m], y_train[:m])
/tmp/ipykernel_1044/2946831180.py:4: DataConversionWarning: A column-vector y wa

***Abbruch nach 4h***
- Auswertung ob Over- oder Underfitting nicht möglich
----

### Interpreation - LIME
#### Erklärung für einzelne Instanzen
- LIME begründet für jede Testinstanz, warum der trainierte Lerne seine Vorhersage gemacht hat

In [34]:
# Imports
from __future__ import print_function

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline


from lime import lime_text
from lime.lime_text import LimeTextExplainer

In [35]:
cols_ratio = ['Rating', 'Rating_Count','Price']
cols_target = ['Maximum_Installs']

dataSet_ratio = numeric.loc[:, cols_ratio]
dataSet_target = numeric[cols_target]

In [36]:
train, test, labels_train, labels_test = train_test_split(dataSet_ratio, dataSet_target, test_size = 0.2, random_state = 42)

In [37]:
rf = RandomForestRegressor(n_estimators=1000)

rf.fit(X_train, y_train)

/tmp/ipykernel_2784/775629154.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RandomForestRegressor(n_estimators=1000)

In [38]:
print('Random Forest MSError', np.mean((rf.predict(X_test) - y_test) ** 2))

ValueError: Unable to coerce to Series, length must be 1: given 448335

In [39]:
print('MSError when predicting the mean', np.mean((y_train.mean() - y_test) **2))

MSError when predicting the mean Maximum_Installs    7.246809e+14
dtype: float64


In [41]:
categorical_features = np.argwhere(np.array([len(set(dataSet_ratio[:,x])) for x in range(dataSet_ratio.shape[1])]) <= 10).flatten()

TypeError: '(slice(None, None, None), 0)' is an invalid key